In [516]:
#%pylab
#import scipy.interpolate
def get_radius_mm(theta):
    """
    Returns the radius in mm given an radius in degrees using the platescale data 
    relative to the center of the focal plane as (0,0)
    Parameters
    ----------
    theta: A float that represents the angle from the center of the focal plane
    """
    platescale = desimodel.io.load_platescale()
    # Uses a quadratic one-dimensional interpolation to approximate the radius in degrees versus radius in mm
    fn = scipy.interpolate.interp1d(platescale['theta'], platescale['radius'], kind = 'quadratic')
    radius = float(fn(theta))
    return radius
def get_radius_deg(x, y):
    """
    Returns the radius in degrees given x, y coordinates using the platescale data
    Parameters
    ----------
    x: The x coordinate in mm of a location on the focal plane
    y: The y coordinate in mm of a location on the focal plane
    """
    radius = np.sqrt(x**2 + y**2)
    platescale = desimodel.io.load_platescale()
    # Plots are used for debugging. 
    #Radius vs theta seems to fit a linear curve better than a quadratic curve
    #plot(platescale['radius'], platescale['theta'], 'k.')
    #plot(platescale['radius'], platescale['radial_platescale'], 'k.')
    fn = scipy.interpolate.interp1d(platescale['radius'], platescale['theta'], kind = 'quadratic')
    degree = float(fn(radius))
    return degree

def cartesian_to_polar_angle(x, y):
    """
    Given cartesian coordinates, this function returns the polar angle in degrees
    for use in polar coordinates
    Parameters
    ----------
    x: The x coordinate in mm of a location on the focal plane
    y: The y coordinate in mm of a location on the focal plane
    """
    # Prevents division by 0
    if(x == 0):
        if(y > 0):
            return 90
        else:
            return 270
    # Case for quadrant 1
    elif(x > 0 and y >= 0):
        return arctan(y / x) * 180 / pi
    # Case for quadrants 2 and 3
    elif(x < 0):
        return (arctan(y / x) * 180 / pi) + 180
    # Case for quadrant 4
    elif(x > 0 and y <= 0):
        return (arctan(y / x) * 180 / pi) + 360
    else:
        raise ValueError

In [517]:
degree = get_radius_deg(333.738, 217.766)
degree
radius = get_radius_mm(1.5731300326614939)
radius

398.5010456698936

In [518]:
def plot_focal_plane_deg(): 
    """
    Plots the focal plane in degrees using the gfa Table object read in with the astropy library
    """
    # ONLY USED HERE RIGHT NOW BECAUSE LOAD_GFA() HAS NOT BEEN PUSHED
    #gfa = desimodel.io.load_gfa()
    import desimodel.io
    %pylab
    import os
    from astropy.table import Table
    gfaFile = os.path.join(os.environ['DESIMODEL'], 'data', 'focalplane', 'gfa.ecsv')
    gfa = Table.read(gfaFile, format = 'ascii.ecsv')
    
    """
    # BEGIN WORKING HERE AFTER LUNCH!!!!!!!
    graph = pyplot.polar()
    lines, labels = pyplot.thetagrids(angles = 30, labels = None, fmt = '%d', frac = 1.1)
    print(labels)
    pyplot.show()
    """
    pyplot.polar()
    title('GFA Overhead View in Degrees')
    # Theta is in degrees
    theta = gfa['Q']
    degree = gfa['RADIUS_DEG']
    print(gfa)
   
    for i in range(40):
        thetaradians = theta[i] * pi / 180
        pyplot.polar(thetaradians, degree[i], 'k.')
        
        # Used for debugging to print the angle in radians and the radius in degrees
        #print(str(theta[i]) + ", " + str(degree[i]))
        
                 

In [519]:
plot_focal_plane_deg()

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib
PETAL CORNER       X        ...       Q         RADIUS_DEG    RADIUS_MM  
                   mm       ...    degrees                               
----- ------ -------------- ... ------------- ------------- -------------
    0      0  116.279135121 ...  287.31935812 1.54426530071 390.595017971
    0      1  121.005023896 ... 287.345026987  1.5999816328 405.886737469
    0      2   91.784287159 ...  283.02060759 1.60541232205 407.384285137
    0      3  87.0587074141 ... 282.826719301 1.54995479057 392.150643917
    1      0  313.248421438 ...  323.31935812 1.54426530071 390.595017971
    1      1  325.620626724 ... 323.345026987  1.5999816328 405.886737469
    1      2  307.552931346 ...  319.02060759 1.60541232205 407.384285137
    1      3  295.180417053 ... 318.826719301 1.54995479057 392.150643917
    2      0  390.567457688 ...  359.31935812 1.54426530071 390.595017971
    2      1  40